In [29]:
from PIL import Image

def add_white_border(input_path, output_path, border_size=50):
    # Buka gambar
    img = Image.open(input_path)
    
    # Dapatkan ukuran gambar asli
    width, height = img.size
    
    # Buat gambar baru dengan border putih
    new_width = width + (2 * border_size)
    new_height = height + (2 * border_size)
    
    # Buat gambar baru dengan background putih
    new_img = Image.new('RGB', (new_width, new_height), 'white')
    
    # Tempel gambar asli di tengah
    new_img.paste(img, (border_size, border_size))
    
    # Simpan gambar
    new_img.save(output_path)

# Penggunaan
input_image = "photo.png"  # Ganti dengan nama file input Anda
output_image = "photo_border.png"  # Nama file output
border_size = 55  # Ukuran border dalam pixel

add_white_border(input_image, output_image, border_size)

In [40]:
#fix add frame terus crop
from PIL import Image

def add_frame_and_crop(photo_path, frame_path, output_path, frame_width, frame_height, crop_width, crop_height):
    """
    Menambahkan frame dan melakukan cropping pada hasil akhir
    
    Parameters:
    - photo_path: path ke file foto
    - frame_path: path ke file frame
    - output_path: path untuk menyimpan hasil
    - frame_width: lebar frame yang diinginkan (dalam pixel)
    - frame_height: tinggi frame yang diinginkan (dalam pixel)
    - crop_width: lebar hasil akhir setelah crop (dalam pixel)
    - crop_height: tinggi hasil akhir setelah crop (dalam pixel)
    """
    # Buka gambar
    photo = Image.open(photo_path)
    frame = Image.open(frame_path)
    
    # Konversi frame ke RGBA
    if frame.mode != 'RGBA':
        frame = frame.convert('RGBA')
    
    # Resize frame sesuai ukuran yang diinginkan
    frame_resized = frame.resize((frame_width, frame_height), Image.Resampling.LANCZOS)
    
    # Buat gambar baru dengan ukuran frame
    result = Image.new('RGBA', (frame_width, frame_height), (0, 0, 0, 0))
    
    # Hitung posisi tengah untuk foto
    x = (frame_width - photo.size[0]) // 2
    y = (frame_height - photo.size[1]) // 2
    
    # Tempel foto di tengah
    result.paste(photo, (x, y))
    
    # Tempel frame
    result = Image.alpha_composite(result, frame_resized)
    
    # Hitung posisi cropping (dari tengah)
    left = (frame_width - crop_width) // 2
    top = (frame_height - crop_height) // 2
    right = left + crop_width
    bottom = top + crop_height
    
    # Crop gambar
    cropped_result = result.crop((left, top, right, bottom))
    
    # Simpan hasil
    cropped_result.convert('RGB').save(output_path, 'PNG', quality=95)

# Contoh penggunaan:
photo_path = "photo_border.png"
frame_path = "border3.png"
output_path = "final_cropped_photo.png"

# Parameter untuk frame
frame_width = 640
frame_height = 580

# Parameter untuk cropping
crop_width = 620   # Sesuaikan dengan kebutuhan
crop_height = 540   # Sesuaikan dengan kebutuhan

add_frame_and_crop(photo_path, frame_path, output_path, 
                  frame_width, frame_height,
                  crop_width, crop_height)

In [1]:
#fix buat add border and crop and jadi slideshow
from PIL import Image
import os
from moviepy.editor import ImageSequenceClip, concatenate_videoclips

def add_white_border(img, border_size=50):
    """Add white border to an image"""
    width, height = img.size
    new_width = width + (2 * border_size)
    new_height = height + (2 * border_size)
    new_img = Image.new('RGB', (new_width, new_height), 'white')
    new_img.paste(img, (border_size, border_size))
    return new_img

def add_frame_and_crop(photo, frame, frame_width, frame_height, crop_width, crop_height):
    """Add frame and crop the image"""
    if frame.mode != 'RGBA':
        frame = frame.convert('RGBA')
    
    frame_resized = frame.resize((frame_width, frame_height), Image.Resampling.LANCZOS)
    result = Image.new('RGBA', (frame_width, frame_height), (0, 0, 0, 0))
    
    x = (frame_width - photo.size[0]) // 2
    y = (frame_height - photo.size[1]) // 2
    
    result.paste(photo, (x, y))
    result = Image.alpha_composite(result, frame_resized)
    
    left = (frame_width - crop_width) // 2
    top = (frame_height - crop_height) // 2
    right = left + crop_width
    bottom = top + crop_height
    
    return result.crop((left, top, right, bottom))

def create_slideshow(input_dir, frame_path, output_video, fps=0.3, repeat=2):
    """
    Create a high quality slideshow video from photos
    
    Parameters:
    - input_dir: directory containing input photos
    - frame_path: path to the frame image
    - output_video: path for the output video
    - fps: frames per second (default: 0.3 = ~3.3 seconds per frame)
    - repeat: number of times to repeat the slideshow
    """
    # Parameters
    border_size = 55
    frame_width = 640
    frame_height = 580
    crop_width = 620
    crop_height = 540
    
    # Load frame image
    frame = Image.open(frame_path)
    
    # Create temporary directory for processed images
    temp_dir = "temp_processed"
    os.makedirs(temp_dir, exist_ok=True)
    
    # Get all PNG files from input directory
    photo_files = [f for f in os.listdir(input_dir) if f.endswith('.png')]
    photo_files.sort()  # Sort files to ensure consistent order
    
    processed_images = []
    
    # Process each photo
    for i, photo_file in enumerate(photo_files):
        # Open and process image
        img = Image.open(os.path.join(input_dir, photo_file))
        
        # Add white border
        img_with_border = add_white_border(img, border_size)
        
        # Add frame and crop
        final_img = add_frame_and_crop(
            img_with_border, frame,
            frame_width, frame_height,
            crop_width, crop_height
        )
        
        # Save processed image with high quality
        temp_path = os.path.join(temp_dir, f"processed_{i}.png")
        final_img.convert('RGB').save(temp_path, 'PNG', quality=100, optimize=False)
        processed_images.append(temp_path)
    
    # Create video clip
    clip = ImageSequenceClip(processed_images, fps=fps)
    
    # Repeat the clip
    final_clip = concatenate_videoclips([clip] * repeat)
    
    # Write high quality video
    final_clip.write_videofile(
        output_video,
        fps=24,  # Higher FPS for smoother transitions
        codec='libx264',
        bitrate="8000k",
        audio=False,  # No audio needed
        preset='veryslow',  # Highest quality encoding
        threads=4  # Use multiple CPU threads
    )
    
    # Clean up temporary files
    for img_path in processed_images:
        os.remove(img_path)
    os.rmdir(temp_dir)

# Penggunaan
input_directory = "public/storage/amlptrii17@gmail.com/photo"  # Directory containing your photos
frame_path = "border.png"  # Path to your frame image
output_video = "output_slideshow.mp4"  # Output video path
fps = 2  # ~3.3 seconds per frame
repeat = 6  # Mengulang slideshow sebanyak 2 kali

create_slideshow(input_directory, frame_path, output_video, fps, repeat)

Moviepy - Building video output_slideshow.mp4.
Moviepy - Writing video output_slideshow.mp4



Moviepy - Done !
Moviepy - video ready output_slideshow.mp4


In [ ]:
from moviepy.editor import ImageSequenceClip, concatenate_videoclips
from PIL import Image, ImageOps
import os


def resize_with_white_border(image_dir, output_dir, output_video_path, target_width, target_height):
    """
    Resizes images to the given target dimensions with white borders filling the extra space.
    """
    os.makedirs(output_dir, exist_ok=True)

    def resize_and_add_white_border(image_path, output_path, target_width, target_height):
        """
        Resize the original image and pad with white borders to fit into the target dimensions.
        """
        with Image.open(image_path) as img:
            # Resize image to fit inside the target size while maintaining aspect ratio
            img_aspect_ratio = img.width / img.height
            target_aspect_ratio = target_width / target_height

            if img_aspect_ratio > target_aspect_ratio:
                # Image is wider than target ratio - adjust width
                new_width = target_width
                new_height = int(target_width / img_aspect_ratio)
            else:
                # Image is taller than target ratio - adjust height
                new_height = target_height
                new_width = int(target_height * img_aspect_ratio)

            img_resized = img.resize((new_width, new_height), Image.Resampling.LANCZOS)

            # Create a blank white canvas of the target size
            white_canvas = Image.new("RGB", (target_width, target_height), (255, 255, 255))

            # Paste the resized image into the center of the white canvas
            paste_x = (target_width - new_width) // 2
            paste_y = (target_height - new_height) // 2
            white_canvas.paste(img_resized, (paste_x, paste_y))

            # Save the final image
            white_canvas.save(output_path, 'PNG', quality=95)

    # Proses semua gambar dari direktori
    processed_images = []
    valid_extensions = ('.png', '.jpg', '.jpeg')
    image_files = [f for f in os.listdir(image_dir) 
                  if f.lower().endswith(valid_extensions)]

    for i, image_file in enumerate(sorted(image_files)):
        input_path = os.path.join(image_dir, image_file)
        output_path = os.path.join(output_dir, f'bordered_{i}.png')
        
        # Resize dan tambahkan border putih di sekitar
        resize_and_add_white_border(input_path, output_path, target_width, target_height)
        processed_images.append(output_path)

    # Buat video dari gambar dengan FPS 2 detik setiap slide
    if processed_images:
        clip = ImageSequenceClip(processed_images, fps=0.5)  # 0.5 fps berarti 2 detik per gambar
        final_clip = concatenate_videoclips([clip] * 2)  # Ulangi 2 kali
        final_clip.write_videofile(output_video_path, codec='libx264', fps=24)
        print(f"Video berhasil disimpan sebagai {output_video_path}")
    else:
        print("Tidak ada gambar yang diproses")


# Playground dan pengaturan
image_dir = "public\\storage\\amlptrii17@gmail.com\\photo"  # Ganti dengan direktori Anda
output_dir = "processed_images_with_border"
output_video = "output_slideshow.mp4"
target_width = int(input("Atur lebar (px): "))  # Input lebar yang diinginkan
target_height = int(input("Atur tinggi (px): "))  # Input tinggi yang diinginkan

resize_with_white_border(image_dir, output_dir, output_video, target_width, target_height)

In [30]:
from PIL import Image

def add_adjustable_frame(photo_path, frame_path, output_path, frame_width, frame_height):
    """
    Menambahkan frame dengan ukuran yang bisa disesuaikan
    
    Parameters:
    - photo_path: path ke file foto
    - frame_path: path ke file frame
    - output_path: path untuk menyimpan hasil
    - frame_width: lebar frame yang diinginkan (dalam pixel)
    - frame_height: tinggi frame yang diinginkan (dalam pixel)
    """
    # Buka gambar
    photo = Image.open(photo_path)
    frame = Image.open(frame_path)
    
    # Konversi frame ke RGBA
    if frame.mode != 'RGBA':
        frame = frame.convert('RGBA')
    
    # Resize frame sesuai ukuran yang diinginkan
    frame_resized = frame.resize((frame_width, frame_height), Image.Resampling.LANCZOS)
    
    # Buat gambar baru dengan ukuran frame
    result = Image.new('RGBA', (frame_width, frame_height), (0, 0, 0, 0))
    
    # Hitung posisi tengah untuk foto
    x = (frame_width - photo.size[0]) // 2
    y = (frame_height - photo.size[1]) // 2
    
    # Tempel foto di tengah
    result.paste(photo, (x, y))
    
    # Tempel frame
    result = Image.alpha_composite(result, frame_resized)
    
    # Simpan hasil
    result.convert('RGB').save(output_path, 'PNG', quality=95)

# Contoh penggunaan:
photo_path = "photo_border.png"
frame_path = "border3.png"
output_path = "final_photo.png"

# Anda bisa mengubah nilai width dan height di sini
frame_height = 580  # Sesuaikan dengan kebutuhan
frame_width = 640   # Sesuaikan dengan kebutuhan

add_adjustable_frame(photo_path, frame_path, output_path, frame_width, frame_height)

In [50]:
from PIL import Image
from moviepy.editor import ImageSequenceClip
import os

def add_white_border(img, border_size=50):
    """Add white border to an image"""
    width, height = img.size
    new_width = width + (2 * border_size)
    new_height = height + (2 * border_size)
    new_img = Image.new('RGB', (new_width, new_height), 'white')
    new_img.paste(img, (border_size, border_size))
    return new_img

def add_frame_and_crop(photo, frame, frame_width, frame_height, crop_width, crop_height):
    """Add frame and crop the image"""
    if frame.mode != 'RGBA':
        frame = frame.convert('RGBA')
    
    frame_resized = frame.resize((frame_width, frame_height), Image.Resampling.LANCZOS)
    result = Image.new('RGBA', (frame_width, frame_height), (0, 0, 0, 0))
    
    x = (frame_width - photo.size[0]) // 2
    y = (frame_height - photo.size[1]) // 2
    
    result.paste(photo, (x, y))
    result = Image.alpha_composite(result, frame_resized)
    
    left = (frame_width - crop_width) // 2
    top = (frame_height - crop_height) // 2
    right = left + crop_width
    bottom = top + crop_height
    
    return result.crop((left, top, right, bottom))

def process_photos_to_gif(input_dir, frame_path, output_gif, fps=1):
    """
    Process multiple photos and create a GIF
    
    Parameters:
    - input_dir: directory containing input photos
    - frame_path: path to the frame image
    - output_gif: path for the output GIF
    - fps: frames per second for the GIF
    """
    # Parameters
    border_size = 55
    frame_width = 640
    frame_height = 580
    crop_width = 620
    crop_height = 540
    
    # Load frame image
    frame = Image.open(frame_path)
    
    # Create temporary directory for processed images
    temp_dir = "temp_processed"
    os.makedirs(temp_dir, exist_ok=True)
    
    # Get all PNG files from input directory
    photo_files = [f for f in os.listdir(input_dir) if f.endswith('.png')]
    photo_files.sort()  # Sort files to ensure consistent order
    
    processed_images = []
    
    # Process each photo
    for i, photo_file in enumerate(photo_files):
        # Open and process image
        img = Image.open(os.path.join(input_dir, photo_file))
        
        # Add white border
        img_with_border = add_white_border(img, border_size)
        
        # Add frame and crop
        final_img = add_frame_and_crop(
            img_with_border, frame,
            frame_width, frame_height,
            crop_width, crop_height
        )
        
        # Save processed image
        temp_path = os.path.join(temp_dir, f"processed_{i}.png")
        final_img.convert('RGB').save(temp_path, 'PNG', quality=95)
        processed_images.append(temp_path)
    
    # Create GIF using MoviePy
    clip = ImageSequenceClip(processed_images, fps=fps)
    clip.write_gif(output_gif, fps=fps)
    
    # Clean up temporary files
    for img_path in processed_images:
        os.remove(img_path)
    os.rmdir(temp_dir)

# Penggunaan
input_directory = "public/storage/amlptrii17@gmail.com/photo"  # Directory containing your photos
frame_path = "border3.png"  # Path to your frame image
output_gif = "final_animation.gif"  # Output GIF path
fps = 0.000000001  # Frames per second - adjust as needed

process_photos_to_gif(input_directory, frame_path, output_gif, fps)

MoviePy - Building file final_animation.gif with imageio.


In [58]:
from PIL import Image
from moviepy.editor import ImageSequenceClip
import os

def add_white_border(img, border_size=50):
    """Add white border to an image"""
    width, height = img.size
    new_width = width + (2 * border_size)
    new_height = height + (2 * border_size)
    new_img = Image.new('RGB', (new_width, new_height), 'white')
    new_img.paste(img, (border_size, border_size))
    return new_img

def add_frame_and_crop(photo, frame, frame_width, frame_height, crop_width, crop_height):
    """Add frame and crop the image"""
    if frame.mode != 'RGBA':
        frame = frame.convert('RGBA')
    
    frame_resized = frame.resize((frame_width, frame_height), Image.Resampling.LANCZOS)
    result = Image.new('RGBA', (frame_width, frame_height), (0, 0, 0, 0))
    
    x = (frame_width - photo.size[0]) // 2
    y = (frame_height - photo.size[1]) // 2
    
    result.paste(photo, (x, y))
    result = Image.alpha_composite(result, frame_resized)
    
    left = (frame_width - crop_width) // 2
    top = (frame_height - crop_height) // 2
    right = left + crop_width
    bottom = top + crop_height
    
    return result.crop((left, top, right, bottom))

def process_photos_to_gif(input_dir, frame_path, output_gif, fps=0.3):
    """
    Process multiple photos and create a high quality GIF
    
    Parameters:
    - input_dir: directory containing input photos
    - frame_path: path to the frame image
    - output_gif: path for the output GIF
    - fps: frames per second for the GIF (default: 0.3 = ~3.3 detik per frame)
    """
    # Parameters
    border_size = 55
    frame_width = 640
    frame_height = 580
    crop_width = 620
    crop_height = 540
    
    # Load frame image
    frame = Image.open(frame_path)
    
    # Create temporary directory for processed images
    temp_dir = "temp_processed"
    os.makedirs(temp_dir, exist_ok=True)
    
    # Get all PNG files from input directory
    photo_files = [f for f in os.listdir(input_dir) if f.endswith('.png')]
    photo_files.sort()  # Sort files to ensure consistent order
    
    processed_images = []
    
    # Process each photo
    for i, photo_file in enumerate(photo_files):
        # Open and process image
        img = Image.open(os.path.join(input_dir, photo_file))
        
        # Add white border
        img_with_border = add_white_border(img, border_size)
        
        # Add frame and crop
        final_img = add_frame_and_crop(
            img_with_border, frame,
            frame_width, frame_height,
            crop_width, crop_height
        )
        
        # Save processed image with high quality
        temp_path = os.path.join(temp_dir, f"processed_{i}.png")
        final_img.convert('RGB').save(temp_path, 'PNG', quality=100, optimize=False)
        processed_images.append(temp_path)
    
    # Create GIF using MoviePy with high quality settings
    clip = ImageSequenceClip(processed_images, fps=fps)
    
    # Write GIF with high quality settings
    # clip.write_gif(
    #     output_gif,
    #     fps=fps,
    #     program='ffmpeg',  # Using ffmpeg for better quality
    #     opt='OptimizePlus',  # Better optimization
    #     fuzz=10,  # Lower value = better quality but larger file
    #     colors=256,  # Maximum colors allowed in GIF
    #     logger=None  # Suppress progress messages
    # )

    clip.write_videofile("final_animation.mp4", fps=fps, codec='libx264', bitrate="8000k")
    
    # Clean up temporary files
    for img_path in processed_images:
        os.remove(img_path)
    os.rmdir(temp_dir)

# Penggunaan
input_directory = "public/storage/amlptrii17@gmail.com/photo"  # Directory containing your photos
frame_path = "border3.png"  # Path to your frame image
# output_gif = "final_animation_hq.gif"  # Output GIF path dengan nama berbeda
fps = 1  # ~3.3 detik per frame

process_photos_to_gif(input_directory, frame_path, output_gif, fps)

Moviepy - Building video final_animation.mp4.
Moviepy - Writing video final_animation.mp4



Moviepy - Done !
Moviepy - video ready final_animation.mp4


In [ ]:
ffmpeg -i output_slideshow.mp4 -vf "scale=1920:1080, minterpolate='mc_mode=mi':fps=60" -c:a copy output_slideshow_1080p.mp4
ffmpeg -i output_slideshow.mp4 -vf "scale=1920:-1" -b:v 15M -c:a copy output_slideshow_1080p.mp4
ffmpeg -i output_slideshow.mp4 -vf "scale=1920:-1, minterpolate='mc_mode=obmc':fps=60" -c:a copy output_slideshow_1080p.mp4
ffmpeg -i output_slideshow.mp4 -vf "scale=1920:-1:flags=lanczos" -c:a copy output_video_1080p_lanczos.mp4
ffmpeg -i output_slideshow.mp4 -vf "scale=1920:-1, unsharp=luma_msize_x=7:luma_msize_y=7:luma_amount=1.5" -c:a copy output_video_sharpened_AI.mp4
ffmpeg -i output_slideshow.mp4 -vf "scale=620:540, unsharp=luma_msize_x=7:luma_msize_y=7:luma_amount=1:chroma_msize_x=5:chroma_msize_y=5:chroma_amount=1.2" -c:a copy output_video_sharpened_AI.mp4
